# How does window thickness affect the temperature of a room over the course of a typical October day in Massachusetts?

Annie Chu and Annie Tor

ModSim Studio 1

In [4]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

10/30/2018
taking weighted average of 2 temperature based on window/wall ratio
sweeping R values (changing material)

total wall dimensions (length x width x thicKNESS): 5 meters by 5 meters by 0.1651 meters (6.5 in) 
glass thickness = 3 in

dhdt outputs joules
need specific heat (of materials, related to R) in to find temp change. 

NEXT STEPS
1. take outside temp data from internet
2. figure out how to implement it with temp_out
3. test the model
4. find R/C/density info for chosen materials
5. choose materials

specific heat (C) must have units of KJ/Kg * K
density must have units of Kg/m3. 

10/27/18 A Productive Meeting!!!

Refined question: How does the window size of a set wall area affect the temperature of a room over the course of a typical October day?

What we call a typical October day will be based off data for October 15th. 
We will be taking data from the outside and finding the equation of best fit for that data using the interpolate function.
Our model will use the differential equation 

dH/dt of plaster = (outside temperature - inside temperature) * area of plaster wall / R-value of plaster wall

dH/dt of window = (outside temperature - inside tempreature) * area of window / R-value of window

The outside temperature will come from our function based on outside data.
The first value for inside temperature will be hard-coded to typical room temperature, and all following values will be found recursively. 
The area of the plaster wall will come from the difference between a predetermined total wall area and the window area.
The area of the window will be swept across a range of values from 0 to a predetermined total wall area.
The R-value of the wall will be based off typical R-values for exterior walls and windows.

To find the inside temperature, we will convert our dH/dt values from heat energy units to temperature units, then add them to the previous inside temperature.

Since we only want to focus on the change of temperature over a day, t0 will be 0 (midnight) and t_end will be 24 (midnight of the next day).

We assume that the window and the plaster will act independently of each other. 
We are also assuming that there is essentially no effect from the Sun, meaning that no thermal energy coming directly from the Sun will be considered. To justify this assumption, our hypothetical window will be North-facing (away from the Sun).

In [6]:
def make_system(window_length, window_width, window_THICC, total_length, total_width, plaster_THICC, window_R, plaster_R, window_C, plaster_C, window_density, plaster_density):
    init = State(temp_in = 0, temp_out = 0)
    t0 = 0
    t_end = 24
    dt = 0.5
    
    total_area = total_length * total_width
    
    window_area = window_length * window_width
    window_volume = window_area * window_THICC
    
    plaster_area = total_area - window_area
    plaster_volume = plaster_area * plaster_THICC
    
    return System(init=init, 
                  t0 = t0, 
                  t_end = t_end, 
                  dt = dt, 
                  total_area = total_area,
                  window_area = window_area,
                  window_volume = window_volume,
                  plaster_area = plaster_area,
                  plaster_volume = plaster_volume,
                  window_length = window_length,
                  window_width = window_width,
                  window_THICC = window_THICC,
                  total_length = total_length,
                  total_width = total_width,
                  plaster_THICC = plaster_THICC,
                  window_R = window_R,
                  plaster_R = plaster_R, 
                  window_C = window_C,
                  plaster_C = plaster_C, 
                  window_density = window_density, 
                  plaster_density = plaster_density)

In [3]:
def update_func(state, t, system):
    
    temp_in = state
    
    unpack(system)
    
    delt_temp = state.temp_out - state.temp_in

    dHdt_plaster = (plaster_area * delt_temp) / plaster_R  #returns heat in joules per second
    dHdt_window = (window_area * delt_temp) / window_R #returns heat in joules per second
    dTdt_plaster = 1000 * ((plaster_area * delt_temp) / (3600 * plaster_R * plaster_C * plaster_density * plaster_area * plaster_THICC))
    dTdt_window = 1000 * ((window_area * delt_temp) / (3600 * window_R * window_C * window_density * window_area * window_THICC))

    temp_in += dTdt_plaster + dTdt_window
    
    return State(temp_in=temp_in)

In [5]:
system = make_system(1.36, 0.2, 0.00136, 0.00027, 0.036, 0.1, 2, 0.33, 100)

TypeError: make_system() takes 4 positional arguments but 9 were given

In [ ]:
def run_simulation(system, update_func):
    unpack(system)
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    for t in linrange(t0, t_end, dt):
        frame.row[t+dt] = update_func(frame.row[t], t, system)
        
    return frame

In [ ]:
results = run_simulation(system, update_func);

In [ ]:
def plot_results(R,L,E):
    plot(R, 'm-', label = "R")
    plot(L, 'g:', label = "L")
    plot(E, 'b-', label = "E")
    decorate(title = "HIV",
            xlabel = "Time",
            ylabel = "Number of Cells")

In [ ]:
plot_results(results.R,results.L,results.E)